# Exploring Datasets with Python

To work with common files like CSV, JSON, Excel files etc., we will use [Pandas](http://pandas.pydata.org/), _an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language_™. Let's import it!

Reference to Dataset
- https://www.statsmodels.org/stable/datasets/generated/fair.html
- https://www.kaggle.com/clarkchong/fairs-affairs-dataset
- https://rstudio-pubs-static.s3.amazonaws.com/300124_43cd93b264fe4ec182e99d2aff939969.html
- https://fairmodel.econ.yale.edu/rayfair/pdf/1978a200.pdf
> Fair, Ray. 1978. “A Theory of Extramarital Affairs,” Journal of Political Economy, February, 45-61.

In [ ]:
import pandas as pd

Our dataset is given as a CSV file. Pandas provides an easy way to read our file with `read_csv`. The path of the file to read is relative to our notebook file. The path can also be an URL, supporting HTTP, FTP and also S3 if your data is stored inside an AWS S3 Bucket!

In [ ]:
affairs = pd.read_csv('datasets/affairs.csv')

The first thing we will check is the size of our dataset. We can use `info()` to get the number of entries of each column.

In [ ]:
affairs.info()

Now we know how many data is inside our file. Pandas is smart enough to parse the column titles by itself and estimate the data types of each column.

You may be curious how the data looks like. Let's see by using `head()`, which will print the first 5 rows.

In [ ]:
affairs.head()

We can access a column of our dataset by using bracket notation and the name of the column.

In [ ]:
affairs['age'].tail()

For categorical features like `sex`, you can also get the distributions of each value by using `value_counts()`.

In [ ]:
affairs['sex'].value_counts()

But what about numerical values? It definitly makes no sense to count each distinct value. Therefore, we can use `describe()`.

In [ ]:
affairs['age'].describe()

You can also access values like `mean` or `max` directly with the corrsponding methods. Let's see who is the oldest cheater!

In [ ]:
affairs['age'].min()

This works for the whole dataframe as well. Pandas knows which values are numerical based on the datatype and hides the categorical features for you.

In [ ]:
affairs.describe()

There is also an easy way to filter your dataset. Let's say we want to have a subset of our data containing only woman. This is also possible with the bracket notation!

In [ ]:
affairs[affairs['sex'] == 'female'].head()

The above statement returns a new dataframe (not a copy, modifying this data will modify the original as well), which can be accessed like before. Let's see how the numerical distribution is for our females.

In [ ]:
affairs[affairs['age'] == 17.5].head()

We can also create new rows. Specify the new column name in brackets and provide a function to set the data. We will create a new column containing True or False, wheather or not the person is below 30.

In [ ]:
affairs['below_30'] = affairs['age'] < 30

In [ ]:
affairs['below_30'].value_counts()

In [ ]:
affairs.head()

We can use this to normalize our columns with better values. Take for example `religious`. The number have the following meaning: 1 = not, 2 = mildly, 3 = fairly, 4 = strongly. We can easily replace them inline with the following code.

In [ ]:
rel_meanings = ['not', 'mildly', 'fairly', 'strongly']

In [ ]:
affairs['religious'] = affairs['religious'].apply(lambda x: rel_meanings[min(x, 4)-1])

In [ ]:
affairs.head()

This should be enought about Pandas. Let's get some visualisations!

## Visualize Data

To visualize our data, we will use [Seaborn](https://seaborn.pydata.org), a Python visualization library based on matplotlib. It provides a high-level interface for drawing attractive statistical graphics. Let's import it.

In [ ]:
import seaborn as sns

To see our charts directly in our notebook, we have to execute the following:

In [ ]:
%matplotlib inline
sns.set()
sns.set_context('talk')

Seaborn together with Pandas makes it pretty easy to create charts to analyze our data. We can pass our Dataframes and Series directly into Seaborn methods. We will see how in the following sections.

### Univariate Plotting

Let's start by visualizing the distribution of the age our our people. We can achieve this with a simple method called `distplot` by passing our series of ages as argument.

In [ ]:
sns.distplot(affairs['age'])

The chart above calculates a kernel density as well. To get a real histogram, we have to disable the `kde` feature. We can also increase to number of buckets for our histogram by setting `bins` to a higher number.

In [ ]:
sns.distplot(affairs['age'], bins=50, rug=True, kde=False)

Interesting! The ages of the people in this dataset seem to end with two or seven.

We can do the same for every numerical column, e.g. the years of marriage.

In [ ]:
sns.distplot(affairs['ym'], bins=10, kde=False)

The average age of our people is around 32, but the most people are married for more than 14 years!

### Bivariate Plotting

Numbers get even more interesting when we can compare them to other numbers! Lets start comparing the number of years married vs the number of affairs. Seaborn provides us with a method called `jointplot` for this use case.

In [ ]:
sns.jointplot(affairs['ym'], affairs['nbaffairs'])

To get a better feeling of how the number of affairs is affected by the number of years married, we can use a regression model by specifying `kind` as `reg`.

In [ ]:
sns.regplot(affairs['ym'], affairs['nbaffairs'])

We can also use a kernel to kompare the density of two columns against each other, e.g. `age` and `ym`.

In [ ]:
sns.jointplot(affairs['ym'], affairs['age'], kind='kde', shade=True)

We can get an even better comparison by plotting everything vs everything! Seaborn provides this with the `pairplot` method.

In [ ]:
sns.pairplot(affairs, dropna=True)

You won't see any special in this data. We need to separate them by some kind of criteria. We can use our categorical values to do this! Seaborn uses a parameter called `hue` to do this. Let's separate our data by `sex` first. To make things even more interesting, let's create a regression for every plot, too!

In [ ]:
sns.pairplot(affairs.drop('below_30', axis=1), hue='sex', kind='reg')

To get even better separation, we can use `lmplot` to compare just the fields we need.

Let's say we're interested in the number of affairs vs years married. We also whant to separate them by `sex`, `child` and `religious`. We will use `sns.lmplot(x="ym", y="nbaffairs", hue="sex", col="child", row="religious", data=affairs)` to achieve this.

In [ ]:
sns.lmplot(x="ym", y="nbaffairs", hue="sex", col="child", row="religious", data=affairs)

Here are some categorical plots to explore the dataset even further.

In [ ]:
sns.boxplot(x="sex", y="ym", hue="child", data=affairs);

In [ ]:
sns.violinplot(x="religious", y="nbaffairs", hue="sex", data=affairs, split=True);

We can also get the correlations between the values by using Pandas builtin method `corr()`.

In [ ]:
affairs.corr()

Feed these stats into Seaborns `heatmap` method will provide us with the visual representation.

In [ ]:
sns.heatmap(affairs.corr(), cmap='coolwarm')